In [1]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on, splink_datasets

In [2]:
db_api = DuckDBAPI()

In [3]:
df = splink_datasets.fake_1000

In [4]:
df.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
0,0,Robert,Alan,1971-06-24,NaN,robert255@smith.net,0
1,1,Robert,Allen,1971-05-24,NaN,roberta25@smith.net,0


In [5]:
settings = SettingsCreator(
    link_type = "dedupe_only",
    comparisons=[
        cl.NameComparison("first_name"),
        cl.JaroAtThresholds("surname"),
        cl.DateOfBirthComparison(
            "dob",
            input_is_string=True,
        ),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        cl.EmailComparison("email"),
    ],
    blocking_rules_to_generate_predictions=[
        block_on("SUBSTRING(first_name, 1, 1)", "dob"),
        block_on("SUBSTRING(surname, 1, 1)"),
    ]
)

In [8]:
linker = Linker(df, settings, db_api)

In [9]:
linker.profile_columns(['first_name', 'city'], top_n=10, bottom_n=5)

AttributeError: 'Linker' object has no attribute 'profile_columns'

In [40]:
linker.training.estimate_probability_two_random_records_match(
    [block_on("SUBSTRING(first_name, 1, 1)", "SUBSTRING(surname, 1, 1)")],
    recall = 0.7
)

Probability two random records match is estimated to be  0.00407.
This means that amongst all possible pairwise record comparisons, one in 245.89 are expected to match.  With 499,500 total possible comparisons, we expect a total of around 2,031.43 matching pairs


In [41]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e6)

You are using the default value for `max_pairs`, which may be too small and thus lead to inaccurate estimates for your model's u-parameters. Consider increasing to 1e8 or 1e9, which will result in more accurate estimates, but with a longer run time.
----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [43]:
linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("SUBSTRING(first_name, 1, 1)", "SUBSTRING(surname, 1, 1)")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(SUBSTRING(l.first_name, 1, 1) = SUBSTRING(r.first_name, 1, 1)) AND (SUBSTRING(l.surname, 1, 1) = SUBSTRING(r.surname, 1, 1))

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Iteration 1: Largest change in params was -0.246 in probability_two_random_records_match
Iteration 2: Largest change in params was -0.0916 in probability_two_random_records_match
Iteration 3: Largest change in params was -0.0721 in the m_probability of email, level `All other comparisons`
Iteration 4: Largest change in params was -0.0607 in the m_probability of email, level `All other comparisons`
Iteration 5: Largest change in params was -0.0391 in the m_probability of email, level `All other comparisons`
Iteration 6: La

<EMTrainingSession, blocking on (SUBSTRING(l.first_name, 1, 1) = SUBSTRING(r.first_name, 1, 1)) AND (SUBSTRING(l.surname, 1, 1) = SUBSTRING(r.surname, 1, 1)), deactivating comparisons first_name, surname>

In [44]:
linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("first_name", "surname")
)


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
(l."first_name" = r."first_name") AND (l."surname" = r."surname")

Parameter estimates will be made for the following comparison(s):
    - dob
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - first_name
    - surname

Level Jaro-Winkler >0.88 on username on comparison email not observed in dataset, unable to train m value

Iteration 1: Largest change in params was -0.0255 in the m_probability of email, level `Jaro-Winkler >0.88 on username`
Iteration 2: Largest change in params was -0.00255 in probability_two_random_records_match
Iteration 3: Largest change in params was -0.000957 in probability_two_random_records_match
Iteration 4: Largest change in params was -0.000347 in probability_two_random_records_match
Iteration 5: Largest change in params was -0.000123 in probability_two_random_records

<EMTrainingSession, blocking on (l."first_name" = r."first_name") AND (l."surname" = r."surname"), deactivating comparisons first_name, surname>

In [45]:
linker.training.estimate_parameters_using_expectation_maximisation(block_on("email"))


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l."email" = r."email"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - dob
    - city

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - email

Iteration 1: Largest change in params was -0.39 in the m_probability of surname, level `Exact match on surname`
Iteration 2: Largest change in params was 0.0811 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0351 in the m_probability of surname, level `All other comparisons`
Iteration 4: Largest change in params was 0.0117 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00485 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00226 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.00112

<EMTrainingSession, blocking on l."email" = r."email", deactivating comparisons email>

In [46]:
pairwise_predictions = linker.inference.predict(threshold_match_weight = -5)

Blocking time: 0.01 seconds
Predict time: 1.26 seconds


In [47]:
clusters = linker.clustering.cluster_pairwise_predictions_at_threshold(
    pairwise_predictions, 0.95
)

Completed iteration 1, num representatives needing updating: 3
Completed iteration 2, num representatives needing updating: 0


In [48]:
df_clusters = clusters.as_pandas_dataframe()

In [49]:
deduped_df = df_clusters.drop_duplicates(subset=['cluster_id'], keep = 'first')

In [50]:
deduped_df

,cluster_id,unique_id,first_name,surname,dob,city,email,cluster
0,4,4,Grace,None,1997-04-26,Hull,grace.kelly52@jones.com,1
1,8,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net,3
2,14,14,Oliver,Griffiths,1991-10-26,Lunton,o.griffiths90@reyes-coleman.com,5
3,22,24,Thoas,Green,1974-10-05,London,thomas.green@clark.org,10
4,26,26,Thomas,Gabriel,1976-09-15,Loodon,gabriel.t54@nnichls.info,11
...,...,...,...,...,...,...,...,...
973,735,735,Georgi,None,2017-05-13,London,None,186
974,571,571,None,Turner,1997-04-24,Glasgow,None,140
990,421,421,Alfie,None,1995-12-10,London,ar20@miller-torres.org,110
993,744,744,Freddie,Carr,1992-08-01,Btournemoth,fcarr22@cook.com,189


In [32]:
df_clusters.shape

(1000, 8)

In [33]:
deduped_df.shape

(605, 8)

In [51]:
deduped_df.shape

(446, 8)

In [52]:
cluster_counts = df_clusters.groupby('cluster_id').size()
nonunique_clusters = cluster_counts[cluster_counts > 1].index
nonunique_df_clusters = df_clusters[df_clusters['cluster_id'].isin(nonunique_clusters)]

In [53]:
nonunique_df_clusters

,cluster_id,unique_id,first_name,surname,dob,city,email,cluster
1,8,9,Evie,Dean,2015-03-03,Pootsmruth,evihd56@earris-bailey.net,3
3,22,24,Thoas,Green,1974-10-05,London,thomas.green@clark.org,10
4,26,26,Thomas,Gabriel,1976-09-15,Loodon,gabriel.t54@nnichls.info,11
5,26,30,Thomas,Gabriel,1976-09-15,London,gabriel.t54@nlchois.info,11
6,37,37,Theodore,Morris,1978-08-19,Birmingham,t.m39@brooks-sawyer.com,13
...,...,...,...,...,...,...,...,...
994,919,922,Evie,Jones,2002-07-22,None,eviejones@brewer-sparks.org,230
995,954,954,Oliver,Roberton,2013-01-06,None,orobertson91@olson-palmer.com,239
996,899,904,Aria,Cross,2000-01-14,Newcastle-upon-Tyne,a.cross58@browning-nguyen.com,224
997,985,990,Nancy,Williasm,1972-01-18,None,None,247
